# Generisches Programmieren mit Java

#### Marcel Lüthi
#### Departement Mathematik und Informatik <br/> Universität Basel

# Agenda

* Problemmotivation
* Lösung: Generische Programmieren mit Objekten (Old Style)
* Intermezzo: Hüllenklassen
* Vergleichen von Generischen Objekten
* Mixed bags


# Ausgangslage

> Problem: Angenommen wir haben folgenden Stack implementiert

In [29]:

class IntStack {
    private int[] data = new int[1000]; // Vereinfachung: Feste grösse
    private int nElements = 0; // Anzahl Elemente im Stack
    
    public void push(int i) {
        data[nElements] = i;     
        nElements += 1;
    }
    
    public int pop() { 
        if (nElements <= 0) {
            throw new RuntimeException("pop from empty stack");
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Client Code

In [30]:
IntStack stack = new IntStack();
stack.push(10);
System.out.println("Size " + stack.size());
System.out.println(stack.pop());
System.out.println("Size " + stack.size());


Size 1
10
Size 0


# Problem

* Was machen wir wenn wir nun einen Stack von Strings wollen?
* ... und einen Stack von Double?
* ... und einen Stack von Floats?

> Kopieren von Code ist keine Lösung!

# Generics old Style

# Lösung

* Ausnutzen, dass alle Klassen von Objekt erben

![hierarchy](./images/class-hierarchy.gif)
<figcaption style="font-size:xx-small">Quelle: https://docs.oracle.com/javase/tutorial/figures/java/classes-object.gif</figcaption>

# Generisches Stack Implementation

In [6]:
class Stack { 
    
    private Object[] data = new Object[1000]; // Vereinfachung: Feste grösse
    private int nElements = 0; // Anzahl Elemente im Stack
 
    public void push(Object element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public Object pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}


# Nutzung des Stacks


In [4]:
// Strings
Stack stringStack = new Stack();
stringStack.push(new String("hello"));
System.out.println(stringStack.pop());


hello


In [35]:
class MyClass {}
Stack myClassStack = new Stack();
myClassStack.push(new MyClass());

# Intermezzo: Hüllenklassen und autoboxing

# Stack mit Primitiven Datentypen

Beobachtung: Wir können nur Objekte auf dem Stack speichern.

* Ist int ein Objekt?
* Funktioniert folgender Code?

In [24]:
int anInt  = 3;
Stack stack = new Stack();
stack.push(anInt);

# Hüllenklassen für Primitive Datentypen

* Zu jedem primitiven Datentyp existiert eine Hüllenklasse (Wrapper)

| Primitiver Typ | Hüllenklasse |
|----------------|--------------|
| int | Integer |
| short | Short |
| long | Long | 
| double | Double |
| float | Float | 
| char | Character | 
| boolean | Boolean |

Beispiel:

In [14]:
Integer anInt = new Integer(5);
int aPrimitiveInt = anInt.intValue();    

# Autoboxing

* *Boxing*: Ein primitiver Typ wird automatisch in ein Objekt der Hüllenklasse konvertiert
* *Unboxing*: Ein Hüllenklassentyp wird automatisch in einen primitiven Typen konvertiert


In [16]:
int aPrinitiveInt = new Integer(5);
Integer aBoxedInt = 5;

# Gotcha: Gleichheit

* Boxing macht Equality noch etwas schwieriger.

In [23]:
System.out.println(new Integer(5) == new Integer(5));
System.out.println(5 == new Integer(5));
System.out.println(new Integer(5) == 5);
System.out.println(new Integer(5).equals(5));



false
true
true
true


# Stacks mit verschiedenen Typen

* Alle Klassen sind Subklassen von Objekt!
    * Wir können Objekte von beliebigen Typen speichern

In [80]:
Stack createMixedStack() { 

    Stack stack = new Stack();
    stack.push("a String");
    stack.push(new Integer(5));
    stack.push(new java.util.Date());
    stack.push(java.awt.Color.GREEN);

    return stack;
}

createMixedStack();


REPL.$JShell$26$Stack@4424ac3f

# Ausdrucken aller Elemente

In [81]:
Stack stack = createMixedStack();
while (stack.size() > 0) { 
    System.out.println(stack.pop());
}

java.awt.Color[r=0,g=255,b=0]
Thu Nov 22 14:52:39 CET 2018
5
a String


* Möglich dank dynamischem binden (der toString Methode)

# Arbeiten mit Elementen

* Speziellere Operationen brauchen mehr Arbeit

In [87]:
Stack stack = createMixedStack();

while (stack.size() > 0) {
    Object element = stack.pop();
    
    if (element instanceof String) {
        String s = (String) element;
        System.out.println("got string, which starts with the letter " +s.charAt(0));
    }
    else if (element instanceof Date) {
        Date d = (Date) element;
        System.out.println("The year is "+ d.getYear());
    }
    else { 
        System.out.println("I don't know what to do with this type of object");
    }
}


I don't know what to do with this type of object
The year is 118
I don't know what to do with this type of object
got string, which starts with the letter a


# Operationen auf Kollektionen von generischen Objekten   

# Hilfsfunktion

In [88]:
Object[] createArrayFromObjects(Object o1, Object o2, Object o3) { 
    Object[] a = new Object[3];
    a[0] = o1; 
    a[1] = o2;
    a[2] = o3;
    return a;    
}

* Erzeugt Array von beliebigen Objekten

# Beispiel 1: Suchen in Objekt

In [89]:

int findObjectInArray(Object[] haystack, Object needle) { 
    for (int i = 0; i < haystack.length; i++) { 
        if (haystack[i].equals(needle)) {
            return i;
        }
    }
    return -1; // not found
}

Object[] array = createArrayFromObjects("String 1", new Date(1970, 12, 31), "String 2" );
findObjectInArray(array, new Date(1970, 12, 31));



1

# Beispiel 2: Sortieren von Arrays

* Sortieren heisst, wir müssen Objekte vergleichen können
    * Was ist grösser, "abc" oder "def"?
    * Was ist grösser, ```Color.GREEN``` oder ```Color.RED```    
* Wir können nur vergleichbare Objekte sortieren.
    * Vergleichbar heisst in Java, sie erben von ```Comparable```
    

In [162]:
Object o1 = new Integer(5);
Object o2 = "a String";
o1 < o2;

CompilationException: 

In [198]:
interface CanCompare { 
   // returns -1, 0, 1
    int compareTo(Object o);
}



In [199]:

public class Tuple implements CanCompare { 

    int first;
    int second;

    public Tuple (int first, int second) {
        this.first = first;
        this.second = second;
    }
    
    public String toString() { 
        return "(" +this.first + ", " + this.second +")";
    }

    public int compareTo(Object thatObject) { 
       if (thatObject instanceof Tuple == false) { 
            throw new IllegalArgumentException("can only compare to Tuples");
        }
        Tuple that = (Tuple) thatObject;
        if (this.first < that.first 
            || this.first == that.first && this.second < that.second) {    
            return -1;
        }
        else if  (this.first == that.first && this.second == that.second) {
            return 0;
        } else { 
            return 1;
        }
    }
}

In [202]:
void sort(CanCompare[] arr) { 
    for (int i = 0; i < arr.length ; ++i) {
        int j = i;
        while(j > 0) {                
            if (arr[j].compareTo(arr[j-1]) < 0) {            
                CanCompare temp = arr[j];
                arr[j] = arr[j-1];
                arr[j-1] = temp;
            }
            j--;
        }
    }    
}


In [203]:
CanCompare[] array = {new Tuple(5,1), new Tuple(1,7), new Tuple(2,8), new Tuple(2,6)};
sort(array);
for (int i = 0; i < array.length; ++i) {
    System.out.println(array[i]);
}

(1, 7)
(2, 6)
(2, 8)
(5, 1)


# Probleme von generischen Stacks:

* Wir brauchen explizite downcasts
* Intent nicht klar
    * Speichern wir Objekte oder Listen oder strings?
* Nicht typesafe
    * Fehler zur Laufzeit - Keine Hilfe von Compiler
* Klassen müssen in Hierarchie gebracht werden
    * Nicht möglich ohne Source Code

# Ausblick

* Bessere Lösung mit Java Generics
    * Teil von Java seit Java 5
* Wichtiges Element von fast allen Programmiersprachen